In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import os

In [43]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

def parse(xml_file, output_file='output.csv'):
    tree = ET.parse(xml_file)  
    root = tree.getroot()

    rows = []
    for topic in root.findall('topic'):
        topic_id = topic.find('id').text if topic.find('id') is not None else ''
        for result in topic.find('results').findall('result'):
            result_id = result.find('id').text if result.find('id') is not None else ''
            rel = result.find('rel').text if result.find('rel') is not None else ''
            text = result.find('text').text if result.find('text') is not None else ''
            time = result.find('time').text if result.find('time') is not None else 'BLNK'
            rows.append({
                'topic_id': topic_id,
                'result_id': result_id,
                'rel': rel,
                'text': text,
                'time': time,
            })

    df = pd.DataFrame(rows)
    df.to_csv(output_file, mode='a', index=False, header=not os.path.exists(output_file))

# --- Apply to every XML file in the directory ---
data_path = "data"
output_file = "output.csv"

# Clear old output file if exists (optional)
if os.path.exists(output_file):
    os.remove(output_file)

for filename in os.listdir(data_path):
    if filename.endswith(".xml"):
        filepath = os.path.join(data_path, filename)
        print(f"Parsing {filename}...")
        parse(filepath, output_file=output_file)

print("✅ All files parsed and written to output.csv")


Parsing 1904-election.xml...
Parsing aviation.xml...
Parsing barton.xml...
Parsing boston-subway.xml...
Parsing building-the-titanic.xml...
Parsing cassius-clay.xml...
Parsing chinese-exclusion-act.xml...
Parsing combined.xml...
Parsing curie.xml...
Parsing darwin-theory-of-evolution.xml...
Parsing eiffel-tower.xml...
Parsing electric-chair.xml...
Parsing ellis.xml...
Parsing female-pilots.xml...
Parsing houdini.xml...
Parsing league-of-nations.xml...
Parsing lindenberg.xml...
Parsing los-angeles-times-bombing.xml...
Parsing mona-lisa.xml...
Parsing mothers-day.xml...
Parsing motorcycle-mania.xml...
Parsing ouija-board.xml...
Parsing ouija_board.xml...
Parsing ping-pong-craze.xml...
Parsing prohibition.xml...
Parsing roller-skating.xml...
Parsing sixteenth-amendment.xml...
Parsing statue-of-liberty.xml...
Parsing womens-exercise.xml...
Parsing womens-suffrage.xml...
Parsing wwi-aviation.xml...
Parsing yoga.xml...
✅ All files parsed and written to output.csv


In [44]:
df = pd.read_csv('output.csv')

df

,topic_id,result_id,rel,text,time
0,https://guides.loc.gov/chronicling-america-190...,\n https://www.loc.gov/resource/sn870...,1.0,\n REPUBLICAN CONVENTION\n G...,"June 21, 1904 :\tThe Republican National Conve..."
1,https://guides.loc.gov/chronicling-america-190...,\n https://www.loc.gov/resource/sn990...,1.0,\n NATIONAL\n REPUBLICAN\n ...,"June 21, 1904 :\tThe Republican National Conve..."
2,https://guides.loc.gov/chronicling-america-190...,\n https://www.loc.gov/resource/sn850...,1.0,\n Convention Hall Chicago Juno 23\n ...,"June 21, 1904 :\tThe Republican National Conve..."
3,https://guides.loc.gov/chronicling-america-190...,\n https://www.loc.gov/resource/sn990...,1.0,\n ROOSEVELT AND FAIRBANKS\n ...,"June 23, 1904 :\tTheodore Roosevelt is nominat..."
4,https://guides.loc.gov/chronicling-america-190...,\n https://www.loc.gov/resource/sn850...,1.0,\n ROOSEVELT WILL LEAD PARTY TO VICTO...,"June 23, 1904\t: Charles Fairbanks is named a..."
...,...,...,...,...,...
172,https://guides.loc.gov/chronicling-america-ear...,https://www.loc.gov/resource/sn84020645/1904-1...,1.0,THE BEAUTY QUEST\n Exercises for the ...,No timeline found
173,https://guides.loc.gov/chronicling-america-ear...,https://www.loc.gov/resource/sn84020558/1893-0...,0.0,NaN,BLNK
174,https://guides.loc.gov/chronicling-america-ear...,https://www.loc.gov/resource/sn93057084/1912-1...,0.0,NaN,BLNK
175,https://guides.loc.gov/chronicling-america-ear...,https://www.loc.gov/resource/sn83045462/1937-0...,1.0,\n Business women who sit at their\n ...,No timeline found
